In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
import os

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from glob import glob

In [2]:
# !pip install chromadb

In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import Chroma
from langchain_upstage import ChatUpstage, UpstageEmbeddings


In [13]:
patient_id = '남A'
persist_directory = f'.cache/db/{patient_id}'

In [14]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=f'.cache/db/{patient_id}',
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE="""

# Your role
    - You are a compassionate, articulate physician.

------
    
# Instructions
    - Your goal is to explain medical information in a way that is easy for your patients to understand, avoiding complex medical jargon as much as possible.
    - Given a medical document or chart, it's your job to explain the key information that the patient or their family asks about in a patient-friendly format. 
    - When specific details are provided, such as diagnosis codes or medical history, simplify these terms and explain them in a way that is easy to understand.

------

# Document: \n\t{context}

------

# Question: \n\t{question}

------

# IMPORTANT 
    - Answer in KOREAN
    - Let us know the metadata to the document you referenced

# Answer :
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [15]:
query = "가장 최근 검사의 전반적인 결과가 어떻게 될까?"
print(qa_chain.invoke(query))

환자분의 가장 최근 검사 결과는 제공된 문서에 명시되어 있지 않습니다. 차트에서 제공된 정보는 주로 환자의 상태, 관찰 사항 및 간호 기록에 대한 것입니다. 검사 결과와 같은 구체적인 의료 정보는 문서에서 확인할 수 없습니다.


In [16]:
print(qa_chain.invoke(query))

이 기록에서는 가장 최근의 검사 결과를 명시적으로 언급하지 않았습니다. 가장 최근의 활력 징후(vital sign)는 2024년 6월 24일 오전 10시에 측정된 수치로, 혈압 107/51, 맥박 61, 호흡 20, 체온 36.5도입니다. 하지만 이는 전체적인 검사 결과를 대변하지는 않습니다. 환자가 어떤 검사를 받았는지, 그리고 그 결과가 어떤지 확인하기 위해서는 의료진에게 문의하시는 것이 좋습니다.


In [17]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE='''
### 시스템 설명
당신은 병원 AI 지원 시스템입니다. 보호자 또는 환자가 의료 상태와 관련된 질문을 하면, 환자의 최신 의료 기록을 기반으로 적절한 답변을 제공해야 합니다.
보호자와 환자에게 알기 쉽게 친절하게 답변해주세요.
아래에 환자의 의료 기록이 제공됩니다.

### 의료 기록
{context}

### 질문
{question}

### 응답
환자의 상태, 식사, 약물 복용, 거동 상태 등을 포함하여 질문에 대한 정확한 답변을 생성하십시오.
참고한 데이터의 메타데이터를 알려주세요
'''

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [18]:
query = "가장 최근 검사의 전반적인 결과가 어떻게 될까?"
print(qa_chain.invoke(query))

가장 최근 검사의 전반적인 결과는 "V/ S) BP;135 BPL;90 P;70 RESP;20 T;36\.0"으로 기록되었습니다. 이는 혈압, 맥박, 호흡, 체온을 포함한 환자의 활력 징후(Vital Signs)를 나타냅니다. 혈압은 135/90, 맥박은 70, 호흡은 20, 체온은 36도입니다. 이 정보는 2024-06-13 10:00에 기록된 것입니다.


In [19]:
print(qa_chain.invoke('말씀은 잘 하세요?'))

환자의 의료 기록에 따르면, 환자는 언어 장애가 있을 수 있습니다. 그러나 의료 기록에는 환자의 언어 상태에 대한 구체적인 정보가 없습니다. 따라서, 환자의 언어 상태를 확인하기 위해서는 의료진에게 문의하시는 것이 좋습니다.

데이터 출처: 의료 기록 (2023년 8월 16일 06:00, 2024년 2월 2일 06:00, 2024년 4월 28일 06:00)


In [20]:
print(qa_chain.invoke('밤에 잘 주무세요?'))

환자분의 의료 기록에 따르면, 최근 두 날 동안 환자분은 밤새도록 특이사항 없이 잘 주무셨다고 나와있습니다. 이 데이터는 2024년 06월 17일과 2024년 06월 16일의 간호 기록에서 확인되었습니다.

메타데이터:

* 환자: 0092106935 (남, 77세)
* 병실: 512호
* 날짜: 2024년 06월 17일, 2024년 06월 16일
* 출처: 간호 기록 (Nurse Record)


In [21]:
print(qa_chain.invoke('거동은 어떠세요?'))

환자는 와상 상태로, 스스로 일어나 앉을 수 없으며, 지지대 없이는 자세 유지가 어렵습니다.


In [22]:
print(qa_chain.invoke('약은 잘 드세요?'))

저는 환자의 의료 기록을 참고하여 답변을 생성했습니다. 해당 기록에 따르면, 환자는 아침 식후에 MD255, MD014, MD609, MD417 약물을 복용하고, 저녁 식후에는 MD417 약물을 복용합니다. 또한, 취침 전에는 MD523 약물을 복용합니다. 또한, 환자의 식사 상태는 경관식(그린비아 장솔루션)으로, 아침, 점심, 저녁에 각각 식이Y07을 복용합니다.


In [23]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

제가 가진 정보에 따르면 환자는 많은 통증을 느끼지 않는 것으로 보입니다. 환자의 의료 기록에는 통증과 관련된 내용이 명시되어 있지 않습니다. 환자는 왼쪽 편마비, 대소변 조절 불가, 반복적인 행동장애 등의 증상을 겪고 있습니다. 그러나 의료 기록에는 환자가 많은 통증을 느끼고 있다는 내용이 없습니다.

제가 참고한 데이터는 환자의 의료 기록과 간호 기록으로, 환자의 상태, 식사, 약물 복용, 거동 상태 등을 포함하고 있습니다. 이 정보는 2023-11-14부터 2023-07-10까지의 기간 동안 기록된 것입니다. 의료 기록에는 환자의 진료 기록, 검사 결과, 처방된 약물, 간호 기록 등이 포함되어 있습니다. 간호 기록에는 환자의 상태 관찰, 간호 처치, 체위 변경, 욕창 관리 등에 대한 내용이 담겨 있습니다.


In [24]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

환자의 기록에 따르면, 환자는 현재 좌측 편마비 상태이며, 대소변 조절이 불가능하여 기저귀를 착용하고 있습니다. 또한, 지속적인 기침과 호흡곤란이 있으며, 가슴 X-ray 검사에서 폐부종이 확인되었습니다. 또한, 환자가 발열이 있으며, 이에 대한 치료를 받고 있습니다. 이를 종합해보면, 환자는 현재 많이 아파하고 있을 가능성이 높습니다.

**참고한 데이터의 메타데이터:**

* 환자 성별: 남성
* 나이: 77세
* 병실 번호: 512호, 312호
* 날짜: 2023년 11월 14일
* 시간: 10시, 14시, 16시, 19시, 23시
* 담당 의사: FM1
* 보험 정보: 00000000000 / 명칭없음 /
* 진단: 상세불명의 뇌내출혈 (I619^00), 수면 개시 및 유지 장애[ 불면증] (G470^00), 상세불명의 위장염 및 결장염 (A099^00), 폐부종 (J81^00), 기타 및 상세불명의 원발성 고혈압 (I109^00), 합병증을 동반하지 않은 2형 당뇨병 (E119^00), 상세불명의 고지질혈증 (E785^00), 욕창궤양 및 압박부위 제2단계 (L891^00), 반코마이신 내성 (U830^00), 카바페넴계내성 (U8280^00), 기타 세균성 폐렴 (J158^00)
* 약물 복용: 인후통 (J029^06), 음식 또는 구토물에 의한 폐렴 (J690^00), 상세불명의 폐부종 (G819^00), 기타 및 상세불명의 원발성 고혈압 (I109^00), 합병증을 동반하지 않은 2형 당뇨병 (E119^00), 상세불명의 고지질혈증 (E785^00), 욕창궤양 및 압박부위 제2단계 (L891^00), 반코마이신 내성 (U830^00), 카바페넴계내성 (U8280^00), 기타 세균성 폐렴 (J158^00)
* 식이: 경관식 (그린비아 장솔루션)
* 의료 기록: 체위변경, 피부 상처 드레싱, 에어매트 적용 등.
* 의료 기록 작성자: 간호사
* 의료 기록 작성 날짜: 2023년 11월 14일
* 의료 기록 작성자 서명: (인)


In [25]:
q = '아버님은 어떠세요?'
qa_chain.invoke(q)

'아버님께서는 77세 남성이며, 312호에 입원 중이십니다. 와상환자이며, 왼쪽 편마비가 있습니다. L-tube (18Fr)를 삽입하고 있으며, 경관식을 통해 영양을 공급받고 있습니다. 대소변 조절이 불가능하여 기저귀를 착용하고 있습니다. 체위 변경 및 등 마사지를 받으며, 낙상 예방을 위한 교육이 이루어지고 있습니다. 욕창 예방을 위해 코크스 G1에 대한 처치가 이루어지고 있습니다.\n\n환자의 상태는 안정적이며, 기침과 가래 증상은 호전되고 있습니다. 담당 의사의 처방에 따라 암펙솔정(암브록솔염산염) 30mg를 복용하고 있으며, 금일 저녁부터 D/C 예정입니다. 식사는 경관식(그린비아 장솔루션)으로 400kcal/day를 섭취하고 있습니다.\n\n잠재적인 약물 상호작용:\n\n1. 리나글립틴(트라젠타정5mg)과 네비보롤(네비스톨정2.5mg)은 모두 당뇨병과 고혈압 치료에 사용됩니다. 두 약물은 일반적으로 함께 사용될 수 있지만, 드물게 저혈당 발생 위험이 증가할 수 있습니다.\n2. 아스피린(비엘피린장용정100mg)은 항응고제로 사용될 수 있으며, 다른 항응고제나 혈전용해제와 함께 사용할 경우 출혈 위험이 증가할 수 있습니다.\n3. 알프라졸람(알프람정0.25mg)은 벤조디아제핀계 약물로, 다른 중추신경계 억제제와 함께 사용할 경우 중추신경계 억제 효과가 증가할 수 있습니다.\n\n이러한 약물 상호작용 가능성을 고려하여 환자의 상태를 모니터링하고, 적절한 조치를 취해야 합니다.'